**GEO6361, semaine 9 : Les opérations spatiales sur des images matricielles**

Cette semaine, nous allons nous intéresser à quelques opérations de base à effectuer sur des images matricielles grâce au module **Rasterio**.

## **9. Opérations spatiales avec Rasterio**

### **9.1 Installons et importons les modules requis**

In [ ]:
%%capture
!pip install rasterio

In [ ]:
import rasterio # le module Rasterio https://rasterio.readthedocs.io/en/latest/intro.html
import rasterio.plot # le sous-module de rasterio permettant de visualiser des données
import matplotlib.pyplot as plt # Matplotlib
import numpy as np # NumPY
import json # Json, pour manipuler les GeoJSON comme des dictionnaires Python

### **9.2 Chargement de données matricielles avec RasterIO**

#### **Importons un geotiff**

On importe notre image avec la méthode "open" de raterio :

In [ ]:
# On importe notre image avec la méthode "open" de raterio. Nous obtenons un objet image couleur spécifique à rasterio.
img = rio.open('/content/MOS_CZ_KR_250.tif')

On peut rapidement visualiser cette image avec la fonction **plot.show** de Rasterio :

#### **Explorons notre geotiff**

Nous obtenons un **objet** Rasterio auquel sont associées de nombreuses méthodes

In [ ]:
 # quelle est la taille de l'image (shape) ?
 # quel est le nombre de couches (count) ?
 # quelle est l'enmprise de la couche dans son système de coordonnées (bounds) ?
 #... d'ailleurs, quel est son système de projection (crs) ?
 # quel est le type des données des pixels de l'image (dtypes) ?
 # Largeur, hauteur (width, height)

In [ ]:
 # pour résumer : profile

Les objets RasterIO contiennent en fait des **array NumPy**. Nous pouvons y accéder directement de la manière suivante :

In [ ]:
img_np =  # pour obtenir une array NumPY (read sur l'objet RasterIO)

À partir de là, on peut manipuler ces images comme on le ferait avec des array NumPy (cf. GÉO6149 semaine 5)

In [ ]:
print("Dimension de l'array :", ...) # pour obtenir les dimensions de l'array (shape)
print("La première bande :", ...) # pour accéder à la première bande ([0])
print("Les valeurs min et max des pixel de la bande :", ..., ...) # les valeurs minimale et maximale de la bande (min, max)
print("La valeur du pixel central de la bande :", ...) # pour accéder à tel ou tel pixel de la bande
print("La moyenne des pixels d'une bande :", ...) # pour calculer des statistiques classiques sur la première bande

plt.imshow(img_np[0], cmap='Greys') # pour afficher une bande

Pour sortir une sous-partie de l'image :

In [ ]:
img_cropped =  # Découper une sous-image composé des lignes 200 à 399 et des colonnes 300 à 599
# Type

In [ ]:
# AFficher le résultat
plt.figure(figsize = (10,10))
rasterio.plot.show()

On voit que l'orsque l'on ressort les arrays de leur objet hôte Rasterio, le système de coordonnées est perdu. Nous pouvons le retrouver grâce à la propriété **.transform** :

In [ ]:
tr = img.transform # array décrivant la transformation affine entre les données projetées à l'array NumPY (https://rasterio.readthedocs.io/en/latest/api/rasterio.transform.html)
print(tr) # Si vous voulez aller plus loin, le principe des transformations affines est expliqué ici : https://www.youtube.com/watch?v=E3Phj6J287o

In [ ]:
print(...) # convertir les coordonnées du système NumPY (numéro de ligne, numéro de colonne) vers le système de coordonnées
print(...)

Et nous pouvons effectuer l'**opération inverse** (passer des coordonnées projetées aux numéros de ligne, colonne)

In [ ]:
print(img.index(-907000.0, -933000.0))
print(img.index(-610000.0, -1411000.0))

Afficher l'histogramme de l'image :

In [ ]:
rasterio.plot.show_hist(
    img_cropped,
    bins=50,
    lw=0.0,
    stacked=False,
    alpha=0.3,
    histtype='stepfilled',
    title="Histogramme des trois bandes"
    )

### **9.3. Chargement de geotiff à partir de Earthexplorer**

On peut parcourir et obtenir les données Landsat sur le site: https://earthexplorer.usgs.gov

Quelques paramètres de Landsat:
* LC = Land Cover
* 08 = Satellite Landsat numéro 8
* Path (trajet) et row (ligne) pour la zone capturée
* Date de capture
* Tiers: 1,2,3 (qualité des données) ou RT (real-time)

On peut par la suite explorer les données stockées sur Google et utiliser le lien direct vers une image précise (en fonction des paramètres de landsat, date, etc.)

In [ ]:
# Trouver une image : https://earthexplorer.usgs.gov

# id = 'LC08_L1TP_047026_20150614_20180131_01_T1' # Vancouver
id = 'LC08_L1TP_014028_20211029_20211109_01_T1' # Montréal <- attention, image nocturne

# Reconstruire une URL de type suivant pour chaque bande B4, B3, B2, et B5:
# https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/014/028/LC08_L1TP_014028_20211029_20211109_01_T1/LC08_L1TP_014028_20211029_20211109_01_T1_B4.TIF



In [ ]:
# Extraire des chaines de caractères avec split('_')
lc =
path =
row =
num =

bandes = []


    remote_path =

    # open
    img =

    # read
    img_data =

    # append
    image


In [ ]:
print(image)
print() # dimensions de la couche en pixels
print() # dimension sur l'axe des x (en pixels)
print() # dimension sur l'axe des y (en pixels)

### **9.4. Création d'une composition colorée (pour une analyse NDVI) :**

**Normalisation des bandes :**

In [ ]:
from skimage import  img_as_ubyte
from skimage import exposure

# Création d'une array 3D pour accueillir trois bandes
channels = np.empty(
    [3,
     image[0].shape[0],
     image[0].shape[1]
     ],
     dtype=np.uint8
     )

# On remplit notre array couche par couche
for index, im in enumerate([image[3], image[0], image[1]]): # On passe ici les bandes composant notre image en (fausses) couleurs Near infrared, B5 (red), red, B4 (green), green, B3 (blue).
    stretched = exposure.equalize_hist(im) # facultatif : les valeurs sont normalisées
    channels[index,:,:] = img_as_ubyte(stretched)

**Affichons nos bandes :**

In [ ]:
fig, (axr, axg, axb) = plt.subplots(1,3, figsize=(21,7))
rasterio.plot.show((channels[0]), ax=axr, cmap='Greys', title='Proche infrarouge (B5)')
rasterio.plot.show((channels[1]), ax=axg, cmap='Greys', title='Rouge (B4)')
rasterio.plot.show((channels[2]), ax=axb, cmap='Greys', title='Vert (B3)')
plt.show()

**Exportons le composé au format geotiff :**

In [ ]:
# inspiré de https://geog-312.gishub.org/book/geospatial/rasterio.html
with rasterio.open('/content/mtl_comp.tif',
                   'w',
                   driver='GTiff',
                   height=img.height,
                   width=img.width,
                   count=3,
                   dtype=channels.dtype,
                   crs=img.crs,
                   transform=img.transform,
                   nodata=0.0
                   ) as dst:
        dst.write(channels)
        keys=['2','3','4']
        for index,chan_name in enumerate(keys):
            dst.update_tags(index+1,name=chan_name)

**ou au format jpg (plus léger car compression avec perte) mais on perd les informations spatiales:**

In [ ]:
with rasterio.open('/content/mtl_comp.tif') as infile:
    profile=infile.profile
    raster=infile.read()
    # produisons un jpeg
    profile['driver']='JPEG'
    jpeg_filename = '/content/mtl_comp.jpeg'
    with rasterio.open(jpeg_filename, 'w', **profile) as dst:
        dst.write(raster)

### **9.5. Découpage d'une image ("clipping")**

**Installons Geopandas :**

In [ ]:
import geopandas as gpd
import json
import rasterio.mask

**On ré-importe le composé coloré geotiff :**

In [ ]:
mtl_comp = rasterio.open('/content/mtl_comp.tif')

In [ ]:
plt.figure(figsize = (10,10))
rasterio.plot.show(mtl_comp)

**Importons un masque pour découper l'image matricielle :**

In [ ]:
masque_gdf = gpd.read_file('/content/masque_mtl.geojson')
print(masque_gdf.crs)
masque_gdf = masque_gdf.to_crs(mtl_comp.crs)
masque_gdf.plot()

**Rasterio a besoin d'un GeoJSON, on lui donne donc un GeoJSON :**

In [ ]:
masque = [json.loads(masque_gdf.to_json())['features'][0]['geometry']]

In [ ]:
masque

**Utilisation de la fonction "mask" de Rasterio :**

In [ ]:
mtl_comp_clipped = rasterio.mask.mask(mtl_comp, masque, crop=True)

In [ ]:
mtl_comp_clipped[0][0]

In [ ]:
plt.imshow(mtl_comp_clipped[0][0], cmap='Greys')

### **9.6. Calcul d'un NDVI (Normalized difference vegetation index)**

https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index

Quel est le type de nos données ?

In [ ]:
mtl_comp_clipped[0].dtype

Comme nous allons effectuer des divisions sur nos valeurs de pixels, nous devons effectuer les calculs sur des entiers (et pas des réels)

In [ ]:
b5_clipped = mtl_comp_clipped[0][0].astype('float32') # Convertissons la couche PIR en réels
b4_clipped = mtl_comp_clipped[0][1].astype('float32') # Convertissons la couche rouge en réels

In [ ]:
b5_clipped.dtype

**Calcul du NDVI :**

In [ ]:
# On configure NumPy pour ignorer les divisions par zéro
np.seterr(divide='ignore', invalid='ignore')

In [ ]:
# Calcul de l'indice (avec ou sans NumPy)
mtl_ndvi =

In [ ]:
# Afficher le résultat
plt.figure(figsize = (10,10))
plt.imshow(mtl_ndvi, cmap='viridis', interpolation='bilinear') # https://matplotlib.org/stable/tutorials/colors/colormaps.html
plt.colorbar()

**Visualisation de l'histogramme :**

In [ ]:
# Pour interpréter : https://ipad.fas.usda.gov/cropexplorer/Definitions/spotveg.htm#:~:text=Normalized%20Difference%20Vegetation%20Index%20(NDVI)%3A&text=In%20general%2C%20NDVI%20values%20range,vegetation%20(0.6%20and%20above).

from rasterio.plot import show_hist
show_hist(mtl_ndvi, bins=50, lw=0.0, stacked=False, alpha=0.3, histtype='stepfilled', title="Histogram")